In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Building Recommender Systems Easily with Merlin Models

In this notebook, we are building a Two-Tower model for Item Retrieval task.

### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models

### Feature Engineering with NVTabular

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf
import glob
import gc

import nvtabular as nvt
from nvtabular.ops import *
from example_utils import workflow_fit_transform

from merlin.schema.tags import Tags
from merlin.schema import Schema

import merlin.models.tf as mm
import merlin.models.tf.dataset as tf_dataloader

from merlin.io.dataset import Dataset
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata
from merlin.models.tf.blocks.core.aggregation import CosineSimilarity

import tensorflow as tf

2022-03-16 18:29:51.549407: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-03-16 18:29:51.549550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


First, we define our input and output paths.

In [3]:
train_path = '/workspace/data/train/*.parquet'
test_path = '/workspace/data/test/*.parquet'
output_path = '/workspace/retrieval/processed/'

ETL Workflow

We select only positive interaction rows therefore we remove rows where `click==0` from the dataset with `Filter()` op.

In [4]:
inputs = ["user_id", "item_id", "item_category", "item_shop", "item_brand",
          'user_shops', 'user_profile', 'user_group',
          'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
          'user_geography', 'user_intentions', 'user_brands', 'user_categories', 'click'] \
         >> Filter(f=lambda df: df["click"] == 1)

user_id = inputs["user_id"] >> AddMetadata(tags=[Tags.USER_ID, Tags.USER]) >> Categorify()
item_id = inputs["item_id"] >> AddMetadata(tags=[Tags.ITEM_ID, Tags.ITEM]) >> Categorify()

item_features = inputs["item_category", "item_shop", "item_brand"] \
     >> AddMetadata(tags=[Tags.ITEM]) >> nvt.ops.Categorify()

user_features = inputs['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
    >> AddMetadata(tags=[Tags.USER]) >> nvt.ops.Categorify()

outputs = user_id+item_id+item_features+user_features


workflow_fit_transform(outputs, train_path, test_path, output_path)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


## Building Two-Tower Model

We will use Two-Tower Model for Item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. It is Thus, these systems are often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). You can read more about two stage Recommender Systems [here](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf). In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/Two-tower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)


We use the `schema` object to define our model.

In [5]:
schema = TensorflowMetadata.from_proto_text_file('/workspace/retrieval/processed/train/').to_merlin_schema()

In [6]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

In [7]:
schema.column_names

['user_id',
 'item_id',
 'item_category',
 'item_shop',
 'item_brand',
 'user_shops',
 'user_profile',
 'user_group',
 'user_gender',
 'user_age',
 'user_consumption_2',
 'user_is_occupied',
 'user_geography',
 'user_intentions',
 'user_brands',
 'user_categories']

We expect the label names to be empty.

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [8]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),        
    loss="categorical_crossentropy",  
    samplers=[mm.InBatchSampler()],
    embedding_options = mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)]
)

Let's explain the parameters in the TwoTowerModel():
- no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [9]:
output_path + '/train/*.parquet'

'/workspace/retrieval/processed//train/*.parquet'

In [10]:
batch_size = 4096

train_dl = tf_dataloader.BatchedDataset(
    Dataset(output_path + 'train/*.parquet', part_size="500MB", schema=schema),
    batch_size = batch_size,
    shuffle= True, 
)

test_dl = tf_dataloader.BatchedDataset(
    Dataset(output_path + 'valid/*.parquet', part_size="500MB", schema=schema),
    batch_size = batch_size,
    shuffle = False,
)

In [11]:
opt = tf.keras.optimizers.Adagrad(learning_rate=0.003)
model.compile(opt, run_eagerly=False)

model.fit(train_dl, epochs=5, validation_data=test_dl)

2022-03-16 18:30:21.164795: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


382/384 [============================>.] - ETA: 0s - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3226 - regularization_loss: 0.0000e+00 - total_loss: 8.3226

2022-03-16 18:30:41.000570: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/then/_0/cond/cond/branch_executed/_168


384/384 [==============================] - 25s 41ms/step - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3196 - regularization_loss: 0.0000e+00 - total_loss: 8.3196 - val_recall_at_10: 0.0027 - val_ndcg_10: 0.0012 - val_loss: 7.3590 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.3590
Epoch 2/5
384/384 [==============================] - 14s 36ms/step - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3188 - regularization_loss: 0.0000e+00 - total_loss: 8.3188 - val_recall_at_10: 0.0027 - val_ndcg_10: 0.0012 - val_loss: 7.3582 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.3582
Epoch 3/5
384/384 [==============================] - 14s 36ms/step - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3182 - regularization_loss: 0.0000e+00 - total_loss: 8.3182 - val_recall_at_10: 0.0027 - val_ndcg_10: 0.0012 - val_loss: 7.3576 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.3576
Epoch 4/5
384/384 [==============================] - 14s 36ms/step - recall_at_10: 

#### Save user tower

In [12]:
query_tower = model.retrieval_block.query_block()
query_tower.save('query_tower')

INFO:tensorflow:Assets written to: query_tower/assets


INFO:tensorflow:Assets written to: query_tower/assets


#### Extract and save user features

In [13]:
user_features = model._ensure_unique(Dataset('/workspace/retrieval/processed/train/*.parquet', schema=schema), tag=Tags.USER, id_tag=Tags.USER_ID).compute()

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [14]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,0,0,1,3,1,3,2,1,0,0,0,0
1,1,3,0,0,0,0,0,0,0,400,19,3
2,2,57,5,2,1,2,1,1,2,123,60,80
3,3,12,4,1,1,1,1,1,2,78,19,3
4,4,1909,1,2,1,2,1,1,2,250,5,9


In [15]:
user_features.shape

(214994, 12)

In [16]:
#user_features.to_parquet('user_features.parquet')

#### Extract and save item features

In [17]:
# train path should be the path for the processed parquet files.
item_features = model._ensure_unique(Dataset('/workspace/retrieval/processed/train/*.parquet', schema=schema), tag=Tags.ITEM, id_tag=Tags.ITEM_ID).compute()

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [18]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,0,0,0,0
1,1,140,380,6
2,2,1,4,33
3,3,38,57,0
4,4,11,65,138


In [19]:
item_features.shape

(568847, 4)

In [20]:
#item_features.to_parquet('item_features.parquet')

#### Extract and save item embeddings

In [21]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

INFO:tensorflow:Assets written to: /tmp/tmpyl49n_um/assets


INFO:tensorflow:Assets written to: /tmp/tmpyl49n_um/assets


In [22]:
item_embs_df

,item_id,item_category,item_shop,item_brand,0,1,2,3,4,5,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,0,-0.008491,-0.102309,0.022300,0.071117,-0.092035,-0.070548,...,-0.218282,-0.088533,0.036797,0.038761,0.050474,0.138893,-0.201486,-0.096185,0.079871,-0.174671
1,1,140,380,6,0.028465,-0.068509,0.010098,0.126853,-0.056414,0.012849,...,-0.063118,-0.021072,0.098170,0.017988,-0.151523,0.217274,-0.102843,-0.026918,0.246797,-0.203013
2,2,1,4,33,-0.100939,-0.145017,0.221424,0.132817,-0.235907,0.030855,...,-0.021422,0.010133,0.066813,0.068083,-0.058502,0.083222,-0.190748,0.072131,0.154896,-0.013758
3,3,38,57,0,-0.104699,-0.032289,-0.077467,-0.095468,-0.181202,-0.016096,...,0.109550,0.061822,0.058454,-0.020658,-0.115805,-0.024861,-0.097081,0.015881,0.037807,-0.070464
4,4,11,65,138,-0.090754,0.021666,-0.073284,0.050787,-0.167716,-0.030687,...,-0.212575,0.019838,0.015235,0.188996,-0.073476,0.208819,-0.118849,-0.037156,0.112738,-0.088297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568842,568842,390,213734,77789,-0.061867,-0.053082,-0.025436,0.027425,0.079073,-0.078578,...,-0.053029,0.057309,-0.055367,0.212112,-0.151913,-0.117846,-0.190842,-0.039447,0.204167,-0.156825
568843,568843,137,111,161,-0.022854,-0.131354,-0.111940,0.055027,-0.026103,-0.001985,...,0.015149,0.092450,-0.006351,-0.000202,-0.031469,0.020679,-0.085500,-0.007744,0.187578,-0.043086
568844,568844,543,140774,56005,0.002140,-0.063220,-0.074880,0.067192,-0.149811,0.004937,...,0.024581,0.114247,-0.052914,0.043048,-0.293942,0.130539,-0.200238,0.075903,0.227444,-0.148763
568845,568845,803,142760,17371,0.085203,-0.008900,-0.010582,0.162580,0.104606,-0.102854,...,0.134081,0.096106,0.152253,0.202794,-0.106855,0.149472,-0.096388,0.056596,0.057033,0.039314


In [23]:
item_embeddings = item_embs_df.iloc[:, 4:]

In [24]:
item_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.008491,-0.102309,0.022300,0.071117,-0.092035,-0.070548,-0.054154,-0.280781,-0.054890,-0.169890,...,-0.218282,-0.088533,0.036797,0.038761,0.050474,0.138893,-0.201486,-0.096185,0.079871,-0.174671
1,0.028465,-0.068509,0.010098,0.126853,-0.056414,0.012849,0.013039,-0.187053,0.037774,-0.009335,...,-0.063118,-0.021072,0.098170,0.017988,-0.151523,0.217274,-0.102843,-0.026918,0.246797,-0.203013
2,-0.100939,-0.145017,0.221424,0.132817,-0.235907,0.030855,-0.059277,-0.132030,0.037480,-0.029862,...,-0.021422,0.010133,0.066813,0.068083,-0.058502,0.083222,-0.190748,0.072131,0.154896,-0.013758
3,-0.104699,-0.032289,-0.077467,-0.095468,-0.181202,-0.016096,-0.026478,-0.214329,0.149610,-0.050052,...,0.109550,0.061822,0.058454,-0.020658,-0.115805,-0.024861,-0.097081,0.015881,0.037807,-0.070464
4,-0.090754,0.021666,-0.073284,0.050787,-0.167716,-0.030687,-0.040895,-0.282884,-0.064349,-0.121975,...,-0.212575,0.019838,0.015235,0.188996,-0.073476,0.208819,-0.118849,-0.037156,0.112738,-0.088297


In [25]:
#item_embeddings.to_parquet('item_embeddings')